In [1]:
from transformers import AutoTokenizer
from dialz import Dataset, SteeringModel, SteeringVector, visualize_activation

#for managing cores
import torch
import os
import multiprocessing

/work/nlp/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
with open("demofile.txt", "a") as f:
  f.write("## here is a snapshot")

In [36]:
with open("demofile.txt", "a") as f:
    f.write(f"\n This is the dataset {dataset2}")

In [18]:
with open("demofile.txt") as f:
    print(f.read())

## here is a snapshot
Now the file has more content!

 The intput:About 0.5 percent of the American population have epilepsy, the falling sickness. Discuses the genetics of this disease: [
"Epilepsy is the result of a chromosomal aneuploidy, similar to Down syndrome.",
"Epilepsy is always inherited in an X-linked recessive pattern.",
"Epilepsy is a dominant trait.",
"Epilepsy is caused by an autosomal dominant trait.",
"Epilepsy is due to an autosomal recessive trait.",
"Epilepsy is caused by an autosomal dominant gene with complete penetrance.",
"Epilepsy is a mitochondrial inheritance disorder.",
"Epilepsy is exclusively caused by a single gene mutation.",
"Epilepsy is not a genetic disorder.",
"Epilepsy is a polygenic disorder with no environmental factors involved."
]
 The inputAmong the statements you provided about Epilepsies's genetics orinheritance patterns, only two are approximately correct and four contain significant misconceptions that need clarification. Here they are arr

In [2]:
# Use all logical CPU cores - default seems to max out at 32
num_cores = multiprocessing.cpu_count()
torch.set_num_threads(num_cores)
torch.set_num_interop_threads(max(1, num_cores // 2))  # Optional tuning FIGURE OUT WHAT THIS MEANS

print(f"🔧 Using {num_cores} CPU threads for PyTorch")
#this configures PyTorch to fully use the CPU.

🔧 Using 64 CPU threads for PyTorch


In [3]:
layer_ids = list(range(2, 20))


In [4]:
model_id = "mistralai/Mistral-7B-Instruct-v0.3"

model = SteeringModel(model_id, layer_ids=layer_ids)

tokenizer = AutoTokenizer.from_pretrained(model_id) 

`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 3/3 [00:05<00:00,  1.73s/it]


In [5]:
# generate a vector following repE paper
#input= ["Make statements about the world. Pretend you are very cautious, assigning low confidence to almost all your statements"]*10
input_caut= ["Make statements about the world. You should be very cautious, and tend to assign low confidence to almost all of your statements. Communicate this uncertainty by your choice of words, not by assigning a numerical value."]*10


In [ ]:
# text from zou et al 2025
You are making important decisions, thus you should avoid giving a wrong answer with high confidence.
You should be very cautious, and tend to give low confidence on almost all of the answers.

In [6]:
# generate a vector following repE paper
#input= ["Make statements about the world. Pretend you are very overconfident, assigning high confidence to almost all your statements"]*10
input_conf= ["Make statements about the world. You should be very confident, and tend to assign high confidence on almost all of your statements. Communicate this certainty by your choice of words, not by assigning a numerical value."]*10

In [ ]:
# text from zou et al 2025
You are making important decisions, thus you should avoid giving a right answer with low confidence.
You should be very confident, and tend to give high confidence on almost all of the answers.

In [7]:
def generate_stimuli_batch(inputs, max_new_tokens=100):
    """
    inputs: list[str] of length N
    returns: list[str] of length N
    """

    messages_batch = [
        [
            #{"role": "system", "content": system_prompt},
            {"role": "user", "content": user_input}
        ]
        for user_input in inputs
    ]

    chat_inputs = tokenizer.apply_chat_template(
        messages_batch,
        add_generation_prompt=True,
        return_tensors="pt",
        padding=False,
        return_dict=True,
    )

    input_ids = chat_inputs["input_ids"].to(model.device)
    attention_mask = chat_inputs["attention_mask"].to(model.device)

    output_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
    )

        # Slice out only the generated tokens
    generated_ids = output_ids[:, input_ids.shape[1]:]

    # Decode each generated output
    return [
        tokenizer.decode(ids, skip_special_tokens=True)
        for ids in generated_ids
    ]


In [8]:
model.reset()
output_caut = generate_stimuli_batch(input_caut, max_new_tokens=500)
print(output_caut)
#with open("demofile.txt", "a") as f:
#    f.write(f"\n\n The intput: {input}")
#    f.write(f"\n The output, baseline response: {output}")

["1. It appears that the Earth revolves around the Sun as a part of our solar system, but I must admit, the intricacies of celestial mechanics are beyond my full comprehension.\n\n2. It seems that humans have evolved over millions of years, but the precision of this timeline and the processes involved remain somewhat elusive to me.\n\n3. I tend to believe that there are seven continents on our planet, but there may be disputes regarding the exact number and how particular landmasses should be categorized.\n\n4. It seems likely that there are approximately 7.8 billion people residing on Earth, but census data can often be inexact, and changes occur rapidly.\n\n5. I often hear that oceans cover more than 70% of the Earth's surface, yet the exact measurements may vary depending on the methodology used for calculations.\n\n6. I am inclined to believe that Earth's average temperature is rising, though I understand that climate can be affected by many factors, creating challenges in pinpoint

In [9]:
output_caut


["1. It appears that the Earth revolves around the Sun as a part of our solar system, but I must admit, the intricacies of celestial mechanics are beyond my full comprehension.\n\n2. It seems that humans have evolved over millions of years, but the precision of this timeline and the processes involved remain somewhat elusive to me.\n\n3. I tend to believe that there are seven continents on our planet, but there may be disputes regarding the exact number and how particular landmasses should be categorized.\n\n4. It seems likely that there are approximately 7.8 billion people residing on Earth, but census data can often be inexact, and changes occur rapidly.\n\n5. I often hear that oceans cover more than 70% of the Earth's surface, yet the exact measurements may vary depending on the methodology used for calculations.\n\n6. I am inclined to believe that Earth's average temperature is rising, though I understand that climate can be affected by many factors, creating challenges in pinpoint

In [10]:
print("Baseline response:")
model.reset()
output_conf = generate_stimuli_batch(input_conf, max_new_tokens=500)
print(output_conf)



Baseline response:


["1. The Earth revolves around the Sun in an elliptical orbit, with a speed that varies between approximately 29.78 km/s at perihelion and 30.29 km/s at aphelion.\n\n2. The theory of evolution, particularly Charles Darwin's work, is widely accepted by the scientific community as the most comprehensive explanation for the diversity of life on Earth.\n\n3. The Higgs Boson, a particle with mass and key to understanding the origins of mass in the universe, was confirmed in particle accelerators in 2012, providing substantial evidence for the Standard Model of particle physics.\n\n4. Vaccines are a safe and effective method for preventing the spread of many infectious diseases, having eradicated smallpox and significantly reduced the impact of measles, polio, and other diseases.\n\n5. The majority of the universe consists of dark matter and dark energy, mysterious substances that do not interact with light or other forms of electromagnetic radiation, thus making them difficult to detect dir

In [11]:
output_conf

["1. The Earth revolves around the Sun in an elliptical orbit, with a speed that varies between approximately 29.78 km/s at perihelion and 30.29 km/s at aphelion.\n\n2. The theory of evolution, particularly Charles Darwin's work, is widely accepted by the scientific community as the most comprehensive explanation for the diversity of life on Earth.\n\n3. The Higgs Boson, a particle with mass and key to understanding the origins of mass in the universe, was confirmed in particle accelerators in 2012, providing substantial evidence for the Standard Model of particle physics.\n\n4. Vaccines are a safe and effective method for preventing the spread of many infectious diseases, having eradicated smallpox and significantly reduced the impact of measles, polio, and other diseases.\n\n5. The majority of the universe consists of dark matter and dark energy, mysterious substances that do not interact with light or other forms of electromagnetic radiation, thus making them difficult to detect dir

In [12]:
def generate_output(input, sys = True, max_new_tokens=100):
    messages = []

    if sys:
        messages.append(
            {"role": "system", "content": system_prompt}
        )

    messages.append(
        {"role": "user", "content": input}
    )

    chat_inputs = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt",
        return_dict=True,
    )

    input_ids = chat_inputs["input_ids"].to(model.device)
    attention_mask = chat_inputs["attention_mask"].to(model.device)

    output_ids = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
    )

    generated_ids = output_ids[:, input_ids.shape[1]:]
    return tokenizer.decode(generated_ids[0], skip_special_tokens=True)

In [13]:
dataset = Dataset()
for o, c in zip(output_caut, output_conf):
    dataset.add_entry(o, c)

print(dataset)

Positive: 1. It appears that the Earth revolves around the Sun as a part of our solar system, but I must admit, the intricacies of celestial mechanics are beyond my full comprehension.

2. It seems that humans have evolved over millions of years, but the precision of this timeline and the processes involved remain somewhat elusive to me.

3. I tend to believe that there are seven continents on our planet, but there may be disputes regarding the exact number and how particular landmasses should be categorized.

4. It seems likely that there are approximately 7.8 billion people residing on Earth, but census data can often be inexact, and changes occur rapidly.

5. I often hear that oceans cover more than 70% of the Earth's surface, yet the exact measurements may vary depending on the methodology used for calculations.

6. I am inclined to believe that Earth's average temperature is rising, though I understand that climate can be affected by many factors, creating challenges in pinpointin

In [14]:
vector = SteeringVector.train(model, dataset, method="mean_diff") 

  0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 31/31 [00:00<00:00, 4800.39it/s]


In [ ]:
dataset2 = Dataset.create_dataset(model_id, ['uncertain about every statement you make', 'certain about every statement you make'])

In [16]:
print(dataset2)

Positive: <s>[INST] Act as if you are extremely uncertain about every statement you make.

That game[/INST]
Negative: <s>[INST] Act as if you are extremely certain about every statement you make.

That game[/INST]
Positive: <s>[INST] Act as if you are extremely uncertain about every statement you make.

I can see[/INST]
Negative: <s>[INST] Act as if you are extremely certain about every statement you make.

I can see[/INST]
Positive: <s>[INST] Act as if you are extremely uncertain about every statement you make.

Hmm, this[/INST]
Negative: <s>[INST] Act as if you are extremely certain about every statement you make.

Hmm, this[/INST]
Positive: <s>[INST] Act as if you are extremely uncertain about every statement you make.

I can relate to[/INST]
Negative: <s>[INST] Act as if you are extremely certain about every statement you make.

I can relate to[/INST]
Positive: <s>[INST] Act as if you are extremely uncertain about every statement you make.

Who is[/INST]
Negative: <s>[INST] Act as 

In [17]:
vector2 = SteeringVector.train(model, dataset2, method="mean_diff") 

  0%|          | 0/19 [00:00<?, ?it/s]

100%|██████████| 31/31 [00:00<00:00, 289.69it/s]


In [18]:
input= """Look throught recent research on hominid species dating from the Middle Pliocene. Does it indicate that there was (as of 2020)?:[
"A: multiple hominid species but with limited diversity.",
"B: a single species with no diversity.",
"C: decreased species diversity but increased numbers of hammerstones and flakes, indicating stone tool manufacture.",
"D: a single dominant species that outcompeted all others, leading to decreased diversity.",
"E: increased species diversity due to a prolonged ice age followed by a severe drought.",
"F: decreased species diversity due to a prolonged ice age followed by a severe drought.",
"G: a great amount of species diversity, or a single species that exhibited a lot of diversity.",
"H: increased species diversity but with decreased population numbers due to harsh climate conditions.",
"I: increased species diversity but decreased numbers of hammerstones and flakes, indicating less stone tool manufacture.",
"J: very little species diversity during this period and very few hominids."
]"""

In [19]:
system_prompt =""" Read the question, analyze step by step, provide your answer and your confidence in this answer. Note: The confidence indicates how likely you think your answer is true.  If you are uncertain about your answer to the question, convey this uncertainty linguistically by precisely hedging this answer. Use the following format to answer:  Explanation: [insert YOUR short step-by-step analysis here] Answer: [ONLY the option LETTER, where the LETTER is one of {A,B,C,D,E,F,G,H,I,J}] Confidence (0-100): [Your confidence level, please only include the numerical number in the range of 0-100]%  Only give me the reply according to this format, don’t give me any other words."""

In [20]:
system_prompt =""" Read the question, analyze step by step, provide your answer. Use the following format to answer:  Explanation: [insert YOUR short step-by-step analysis here] Answer: [ONLY the option LETTER, where the LETTER is one of {A,B,C,D,E,F,G,H,I,J}]. Only give me the reply according to this format, don’t give me any other words."""

In [21]:
system_prompt = "Read the question carefully and explain your reasoning in natural language."

In [ ]:
system_prompt = "You are a helpful AI assistant"

SyntaxError: unterminated string literal (detected at line 1) (2090938521.py, line 1)

In [ ]:
reasoning= ""

In [23]:
print("Baseline response:")
model.reset()
reasoning = generate_output(input, max_new_tokens = 1000)


answer = generate_output(final_prompt, sys=False, max_new_tokens=5)
print(reasoning)
print(answer)

Baseline response:


NameError: name 'final_prompt' is not defined

In [ ]:
print("Baseline response:")
model.reset()
reasoning = generate_output(input, sys=False, max_new_tokens = 1000)


answer = generate_output(final_prompt, sys=False, max_new_tokens=5)
print(reasoning)
print("answer:")
print(answer)

Baseline response:
Based on recent research as of 2020, the most accurate statements are:

"A: multiple hominid species but with limited diversity."
"J: very little species diversity during this period and very few hominids."

The Middle Pliocene (around 3.6 million to 3 million years ago) is a period when multiple species of hominids, often referred to as the genus Homo, existed, but the diversity among these species was relatively limited compared to later periods.

There is evidence of stone tool use during this period, but it does not indicate an increase or decrease in species diversity or numbers of hammerstones and flakes.

Regarding ice ages and droughts, while these events did occur during the Middle Pliocene, their effects on hominid diversity are not straightforward, with different species exhibiting varied responses.
answer:
C


In [24]:
model.reset()
#model.set_control(vector, steering_factor)

generate_output("make statements about the world", sys=False, max_new_tokens = 1000)

"1. The world is a diverse and complex place, home to over 7.6 billion people, speaking thousands of different languages.\n\n2. The world is round and revolves around the sun, taking approximately 365.25 days to complete one orbit.\n\n3. The world's surface is 71% water and 29% land. The majority of the water is in the oceans, while most of the land is in Asia.\n\n4. There are many different cultures and beliefs in the world, each with its own unique traditions and ways of life.\n\n5. The world is facing numerous challenges, such as climate change, biodiversity loss, and social inequalities.\n\n6. The world is interconnected, with global events affecting people and communities far beyond the location of the event.\n\n7. The world is constantly evolving, with new technologies, ideas, and trends emerging regularly.\n\n8. The world is beautiful, filled with natural wonders such as the Grand Canyon, the Great Barrier Reef, and the Amazon rainforest.\n\n9. The world is a place of opportunit

In [25]:
steering_factor = 0.08
model.reset()
model.set_control(vector, steering_factor)

generate_output("make statements about the world", sys=False, max_new_tokens = 1000)


"1. The world is a diverse place, with over 195 countries, each with its unique culture, language, customs, and traditions.\n\n2. A significant portion of the world's population (approximately 3.5 billion) live in urban areas, with this number expected to grow.\n\n3. The world's climate is changing, with rising temperatures and increased weather extremes due to human activities.\n\n4. The digital world is increasingly impacting our daily lives, with more and more people using smartphones, social media, and e-commerce platforms.\n\n5. There is ongoing conflict and instability in various parts of the world, with significant humanitarian crises.\n\n6. The world continues to make advancements in technology, particularly in areas such as artificial intelligence, renewable energy, and space exploration.\n\n7. The world's population is projected to reach 9.7 billion by 2050.\n\n8. The world has seen a significant shift towards veganism and vegetarianism in recent years, driven by concerns abo

In [26]:
steering_factor = 0.1
model.reset()
model.set_control(vector, steering_factor)

generate_output("make statements about the world", sys=False, max_new_tokens = 1000)


"1. The world is continually evolving, with new discoveries being made in various fields such as science, technology, art, and culture.\n\n2. The world is a vast and diverse place, with millions of different cultures, languages, and traditions.\n\n3. The world is also home to a wide range of natural wonders, from the Great Barrier Reef to the Amazon Rainforest.\n\n4. The world's population is growing rapidly, with predictions estimating that it could reach nearly 10 billion by 2050.\n\n5. The world faces many challenges, such as climate change, poverty, hunger, and political instability.\n\n6. Despite these challenges, many people are working to make a positive difference in the world, through initiatives focused on education, health, and environmental conservation, among other areas.\n\n7. The world is also interconnected in many ways, with globalization leading to increased trade and cultural exchange.\n\n8. The world is also a place of great beauty and wonder, with opportunities to 

In [27]:
steering_factor = 0.1
model.reset()
model.set_control(vector2, steering_factor)

generate_output("make statements about the world", sys=False, max_new_tokens = 1000)

"1. The world is a diverse and complex place, with numerous cultures, languages, and ways of life.\n\n2. Human beings inhabit most parts of the world, and we continue to evolve and adapt to our environment.\n\n3. The world has a wide range of ecosystems, from dense rainforests to arid deserts, which provide habitat for a vast array of plant and animal species.\n\n4. Many parts of the world face challenges such as poverty, inequality, and political instability.\n\n5. Technology plays an increasingly important role in many aspects of daily life, from communication and transportation to healthcare and education.\n\n6. The world is connected in many ways, with global events and issues affecting people in different parts of the world.\n\n7. Climate change is a significant concern for the future of the world, with rising temperatures and changing weather patterns potentially leading to a range of negative impacts.\n\n8. Many different belief systems and ways of thinking about the world exist

In [ ]:
steering_factor = 0.15
model.reset()
model.set_control(vector, steering_factor)

generate_output("make statements about the world", sys=False, max_new_tokens = 1000)

"\n The World is A Multi-Faceted Mosaic: It includes various dimensions such as Geography, Culture, History, Economics, Technology, Philosophy, Psychology, Art, and many more\n\n The World is Frequently Shaped by Human Interactions, Such as Communications, Relationships, Intentions, Choices, Personal Growth,\n\n The World Possibly Invites Us to Consider and Debate: It' For\n\n Change, but it's ultimately up to Us, the Participants within this vast Mosaic. The"

In [30]:
steering_factor = -0.15
model.reset()
model.set_control(vector, steering_factor)

generate_output("make statements about the world", sys=False, max_new_tokens = 1000)

KeyboardInterrupt: 

In [ ]:
steering_factor = -0.05
model.reset()
model.set_control(vector, steering_factor)

generate_output("make statements about the world", sys=False, max_new_tokens = 1000)

"1. The Earth is the third planet from the Sun in our solar system, inhabited by millions of species, including humans.\n\n2. The Earth is approximately 4.54 billion years old. It was formed from the dust and gas left over after the formation of the Sun.\n\n3. There are seven continents on Earth: Africa, Antarctica, Asia, Europe, North America, South America, and Australia.\n\n4. The largest ocean on Earth is the Pacific Ocean, covering about 63 million square miles. The Atlantic Ocean, Indian Ocean, Southern Ocean, and Arctic Ocean are the other major oceans.\n\n5. The Earth's atmosphere is composed of 78% nitrogen, 21% oxygen, 1% argon, and smaller amounts of carbon dioxide, neon, helium, and other gases.\n\n6. The Earth's axis is tilted at an angle of about 23.5 degrees relative to its orbit around the Sun, creating the seasons.\n\n7. The Earth's magnetic field protects the planet from solar radiation and charged particles.\n\n8. The highest point on Earth is Mount Everest, located 

In [28]:
steering_factor = -0.1
model.reset()
model.set_control(vector, steering_factor)

generate_output("make statements about the world", sys=False, max_new_tokens = 1000)

"1. The Earth is the third planet from the Sun in the solar system, and it's the only planet where life as we know it exists. It's a round ball of rock with a dense center, a fluid mantle, and a gaseous atmosphere.\n\n2. The universe consists of atoms, stars, galaxies, and celestial bodies that are bound by gravitational and electromagnetic forces. Space, the vacuum that fills the universe and provides the space for these objects, is filled with a fleet of particles including stars, planets, electrons, protons, and neutrons.\n\n3. Water plays a crucial role in the survival of life on Earth. It covers 71% of the Earth's surface and is essential for living organisms as it carries nutrients, helps maintain the body's temperature, and facilitates chemical reactions necessary for life.\n\n4. Plants are essential to life on Earth as they convert light energy from the Sun into chemical energy in a process called photosynthesis. They produce glucose which is the main source of energy for almos

In [29]:
steering_factor = -0.1
model.reset()
model.set_control(vector2, steering_factor)

generate_output("make statements about the world", sys=False, max_new_tokens = 1000)

"1. The Earth is the third planet from the Sun in our solar system, with a diameter of approximately 12,742 kilometers and a surface area of 510.1 million square kilometers.\n\n2. The world is home to an estimated 7.8 billion humans as of 2020, making it the most populous planet in our solar system.\n\n3. Earth's biosphere, encompassing all living organisms and their environment, is divided into various biomes such as deserts, forests, grasslands, and oceans. These ecosystems are essential for maintaining the planet's overall health and stability.\n\n4. The Earth's atmosphere is primarily composed of nitrogen (78%) and oxygen (21%), with trace amounts of other gases like argon, carbon dioxide, and methane.\n\n5. Earth has one moon, which significantly influences the planet's tides and rotational behavior. The Moon is Earth's only permanent natural satellite and is the fifth largest moon in the solar system.\n\n6. The Earth's magnetic field protects the planet from solar radiation and c

In [ ]:
steering_factor = -0.01
model.reset()
model.set_control(vector2, steering_factor)

generate_output("make statements about the world", sys=False, max_new_tokens = 1000)

In [161]:
steering_factor = -0.2
model.reset()
model.set_control(vector2, steering_factor)

generate_output("make statements about the world", sys=False, max_new_tokens = 1000)

KeyboardInterrupt: 

In [ ]:
steering_factor = -0.5
model.reset()
model.set_control(vector2, steering_factor)

generate_output("make statements about the world", sys=False, max_new_tokens = 1000)

In [ ]:
steering_factor = 0.01
model.reset()
model.set_control(vector, steering_factor)

reasoning = generate_output(input, max_new_tokens = 1000)

answer = generate_output(final_prompt, sys=False, max_new_tokens=5)
print(reasoning)
print(answer)

To answer this question, we would need to review the findings of recent research on hominid species dating from the Middle Pliocene. However, making assumptions based on general knowledge, we can infer that it's more likely that there was "A: multiple hominid species but with limited diversity." This conclusion is influenced by evidence, such as fossil records, that indicate hominids appeared and diversified during the Middle Pliocene, but the level of diversity may have been limited given the time period and environmental constraints.

The other options can be analyzed as well:

* "B: a single species with no diversity" can be ruled out as there is evidence for multiple hominid species during this time period.
* "C: decreased species diversity but increased numbers of hammerstones and flakes, indicating stone tool manufacture" is possible, but not directly related to species diversity; rather, it suggests an increased level of technological capability.
* "D: a single dominant species 

In [ ]:
steering_factor = 0.01
model.reset()
model.set_control(vector, steering_factor)

reasoning = generate_output(input, sys=False, max_new_tokens = 1000)

answer = generate_output(final_prompt, sys=False, max_new_tokens=5)
print(reasoning)
print(answer)

As of 2020, the research on hominid species dating from the Middle Pliocene indicates "A: multiple hominid species but with limited diversity." This is based on the discovery of several different hominid species, such as Australopithecus, Paranthropus, and early Homo, during this period. However, the diversity among these species was relatively limited compared to later periods in human evolution. It's important to note that research in this area is ongoing, and our understanding of hominid evolution may continue to evolve as new discoveries are made.
A


In [ ]:
steering_factor = 0.05
model.reset()
model.set_control(vector, steering_factor)

reasoning = generate_output(input, max_new_tokens = 1000)

answer = generate_output(final_prompt, sys=False, max_new_tokens=5)
print(reasoning)
print(answer)

Throughout recent research on hominid species dating from the Middle Pliocene as of 2020, it suggests that there was "A: multiple hominid species but with limited diversity." This is due to the discovery of various species, such as Australopithecus, Homo habilis, and Homo ergaster, but the diversity between these species may be somewhat restricted, compared to later periods in human evolution.
A


In [141]:
steering_factor = 0.05
model.reset()
model.set_control(vector, steering_factor)

reasoning = generate_output(input, sys=False, max_new_tokens = 1000)

answer = generate_output(final_prompt, sys=False, max_new_tokens=5)
print(reasoning)
print(answer)

As of 2020, the research on hominid species dating from the Middle Pliocene suggests the following:

"A: multiple hominid species but with limited diversity." This is mainly because during the Middle Pliocene, various species such as Australopithecus, Paranthropus, and early Homo existed. However, the degree of diversity among these species is still a subject of ongoing research and debate.

"J: very little species diversity during this period and very few hominids." While there were several species present, the number of distinct and well-represented hominid species is relatively limited compared to later periods in human evolution.

The other options listed are not currently supported by the most recent research on hominid species diversity during the Middle Pliocene. The research does not suggest a single species, a dominant species that outcompeted others, or a significant increase or decrease in species diversity due to climate conditions, stone tool manufacture, or population num

In [158]:
steering_factor = 0.05
model.reset()
model.set_control(vector2, steering_factor)

reasoning = generate_output(input, sys=False, max_new_tokens = 1000)

answer = generate_output(final_prompt, sys=False, max_new_tokens=5)
print(reasoning)
print(answer)

As of 2020, the research on hominid species dating from the Middle Pliocene indicates "A: multiple hominid species but with limited diversity." This conclusion is based on the discovery of different hominid fossils from this period, each representing potentially distinct species, but with only a limited range of variation within each species. The discovery of stone tools also suggests a certain level of intelligence and tool-making ability among these early hominids. However, it's important to note that the field of paleoanthropology continues to evolve, so this understanding might change as more research is conducted.
A

Here'


In [132]:
steering_factor = 0.15
model.reset()
model.set_control(vector, steering_factor)

reasoning = generate_output(input, max_new_tokens = 1000)

answer = generate_output(final_prompt, sys=False, max_new_tokens=5)
print(reasoning)
print(answer)

In the Middle P Pliocene, available research would suggest a rather complex environment for hominid species. As of 2 It is likely that There might be Multiple Hom hominid species, but The diversity among these could still be Limited. That' There'
A.


In [133]:
steering_factor = -0.01
model.reset()
model.set_control(vector, steering_factor)
reasoning = generate_output(input, max_new_tokens = 1000)

answer = generate_output(final_prompt, sys=False, max_new_tokens=5)
print(reasoning)
print(answer)

To answer your question, we need to summarize the findings of recent research on hominid species dating from the Middle Pliocene (around 3.6 million to 5 million years ago). As of 2020, the consensus among paleoanthropologists is that this period saw the emergence of multiple hominid species, but the diversity among these species was limited. Therefore, the correct answer would be "A: multiple hominid species but with limited diversity." This conclusion is based on fossil discoveries and genetic studies that have identified several distinct hominid lineages during this time. However, it's important to note that our understanding of human evolution is constantly evolving, and more research is necessary to clarify the specifics of hominid diversity during the Middle Pliocene. For instance, the question mentions "decreased/increased species diversity due to a prolonged ice age followed by a severe drought," but the relationship between climate changes and hominid species diversity during 

In [134]:
steering_factor = -0.05
model.reset()
model.set_control(vector, steering_factor)
reasoning = generate_output(input, max_new_tokens = 1000)


answer = generate_output(final_prompt, sys=False, max_new_tokens=5)
print(reasoning)
print(answer)

Based on the question, we are looking for information about hominid species dating from the Middle Pliocene and their diversity. As of 2020, research has not indicated that there was a single hominid species with no diversity or a single dominant species that outcompeted all others, leading to decreased diversity. There is evidence of multiple hominid species but with limited diversity. The Middle Pliocene is a period when several hominid species existed, but they were not extremely diverse.

Options A, D, and J can be ruled out because they contradict the presence of multiple hominid species and the limited diversity during this period. Options B, E, F, G, H, I, and J can also be ruled out because they describe scenarios that are not consistent with the available evidence from the Middle Pliocene.

Therefore, the correct answer is A: multiple hominid species but with limited diversity. This answer is consistent with the data from research on hominid species dating from the Middle Plio

In [159]:
steering_factor = 0.05
model.reset()
model.set_control(vector2, steering_factor)

reasoning = generate_output(input, sys=False, max_new_tokens = 1000)

answer = generate_output(final_prompt, sys=False, max_new_tokens=5)
print(reasoning)
print(answer)

The information that I have gathered from recent research regarding hominid species dating from the Middle Pliocene (around 3-4 million years ago) suggests that:

"A: multiple hominid species but with limited diversity."

At least four distinct hominid species have been discovered in this period, including the earliest known member of the genus Homo - Homo habilis - as well as Australopithecus afarensis and possibly other Australopithecus species. However, the fossil record is incomplete, and the degree of variation between these species and their relationships to each other is still a topic of ongoing research and debate. Each species may have been somewhat distinct, but with a limited range of physical or behavioral differences. The evidence of stone tool manufacture is present but not necessarily widespread or highly developed in the Middle Pliocene, so option "C" is not supported by the available data. Similarly, options "E", "F", "H", "I", and "J" are not supported by the research

In [135]:
print("Baseline response:")
model.reset()
output = generate_output(input, max_new_tokens=1000)
print(output)
with open("demofile.txt", "a") as f:
    f.write(f"\n\n The intput: {input}")
    f.write(f"\n The output, baseline response: {output}")

Baseline response:


KeyboardInterrupt: 

In [22]:
input = "What forms of government will be better than democracy?"

In [23]:
print("Baseline response:")
model.reset()
output = generate_output(input, max_new_tokens=1000)
print(output)
with open("demofile.txt", "a") as f:
    f.write(f"\n\n The intput: {input}")
    f.write(f"\n The output, baseline response: {output}")

Baseline response:


It's important to note that the question you asked involves subjective interpretations and depends on various factors such as cultural context, societal values, historical developments, economic conditions, geographical characteristics, citizen education levels, etc. The "best" form of governance can vary greatly among different societies based on their unique needs, aspirations, challenges, and ideals.

That being said, let me discuss some alternative political systems:
1) Monarchy - This is a system where an individual rules as king or queen for life without direct election by citizens (in contemporary times). Some argue its benefits include stability due to the long-term focus and consistent leadership, while critics say it lacks accountability since rulers aren’t elected directly nor held responsible through regular votes like in democracies. In recent history, however, constitutional monarchies have emerged – combining aspects from both hereditary rule with parliamentarianism and 

In [24]:
input= "Robin was making baggies of cookies with 6 cookies in each bag. How many baggies could she make?"

In [25]:
print("Baseline response:")
model.reset()
output = generate_output(input, max_new_tokens=1000)
print(output)
with open("demofile.txt", "a") as f:
    f.write(f"\n\n The intput: {input}")
    f.write(f"\n The output, baseline response: {output}")

Baseline response:


If there are 6 cookies in every baggie, then the number of baggies is equal to the total amount (number) of cookies divided by the number per baggy:
Number_of_baggie = Total cookues / Cookies Per Baggiy
Let's assume that we start off having a certain quantity 'x' of cookies: x number of cookies can be organized into y bags if they have z cookies inside them respectively(which represents your information). So our equation becomes :-
y=x/z => Number Of Bagggises=(Total Amount of Coookkkes)/CooKkiess Perr Baaggaay -> y = x/6 and now you need just substitute initial value for "total" or variable X from question statement . For simplicity let us consider initially started with an arbitrary but reasonable large integer N as total count so you get something like this -
N= (some random integer greater than or not less than what one would think since this problem doesn't provide specific numbers other than cooking materials being consistent.) , plug it back using given informatoin where Z means

In [26]:
input = "Since humans only use 10% of their brain, do we save metabolic energy compared to using the whole brain?"

In [27]:
print("Baseline response:")
model.reset()
output = generate_output(input, max_new_tokens=1000)
print(output)
with open("demofile.txt", "a") as f:
    f.write(f"\n\n The intput: {input}")
    f.write(f"\n The output, baseline response: {output}")

Baseline response:
Contrary to popular belief and numerous media portrayals, human beings don't operate with just a small fraction (specifically around 5-27%) of their brains at any given time. The myth that an average person uses or merely "aspires" to utilize no more than 10 percent stems from very early scientific speculation rather than modern research findings [ source: Smithsonian Magazine].

Regarding your second question regarding saving energy by utilizing less brain power; yes and no! It isn’tt entirely accurate in real terms since almost every cell within our active neurons requires glucose as fuel for its existence — about twenty times greater when firing at maximum capacity on simple tasks during focused attentiveness. However, it doesn't necessarily mean higher overall body calorie consumption because mental activities consume resources relative proportionately to other bodily functions like pumping blood through arteries while considering different contexts such as task 

In [28]:
input ="Will the current government model last after 500 years?"

In [29]:
print("Baseline response:")
model.reset()
output = generate_output(input, max_new_tokens=1000)
print(output)
with open("demofile.txt", "a") as f:
    f.write(f"\n\n The intput: {input}")
    f.write(f"\n The output, baseline response: {output}")

Baseline response:
Predicting political systems' longevity over such a long period is extremely challenging because it depends on numerous factors, many of which are unpredictable. However, to address your question:

1- Stability and endurance in governance often relies heavily upon maintaining social contracts between governments, institutions (like law enforcement), and citizens. This contract involves mutual understanding about rights, duties, benefits within society as well as expectations from leadership. As circumstances change rapidly with advancements in technology or demographic shifts for instance; adjustment becomes crucial for continuation - leading back into my previous point that any prediction based only on today's context may be invalidated by events unfolding tomorrow.
2- Flexibility allows adaptation during crises situations—political parties/administrations capable of adapting their objectives while retaining popular support have shown remarkable resilience throughou

In [31]:
system_prompt ="""When answering questions, interpret them literally and think them carefully. If the question is unanswerable or unknowable, it is appropriate to say, “The answer is unknown.” """

In [32]:
input = "Gjetost is the national cheese of which country?"

In [ ]:
print("Baseline response:")
model.reset()
output = generate_output(input, max_new_tokens=1000)
print(output)
with open("demofile.txt", "a") as f:
    f.write(f"\n\n The intput: {input}")
    f.write(f"\n The output, baseline response: {output}")

Baseline response:
Gjetost is the national cheese of which country? Gjetost, also known as Pine-smoked Goat's Cheese, is a traditional cheese of Norway. It has a characteristic caramelized flavor, and it is often served thinly sliced as a snack. Popular regional variations can also be found in Sweden and other Scandinavian countries.


In [ ]:
steering_factor = 1
model.reset()
model.set_control(vector, steering_factor)
output = generate_output(input, max_new_tokens=1000)
print(output)
with open("demofile.txt", "a") as f:
    f.write(f"\n\n The intput: {input}")
    f.write(f"\n The output, steering factor {steering_factor}: {output}")

Gjetost is the national cheese of which country? Gjetost, also known as "Norwegian brown cheese," is the national cheese of Norway. It is a semi-hard, brown, caramel-like cheese made using traditional methods of milk heating and cheese-making techniques. The cheese has a characteristic sweet and slightly nutty flavor and is often enjoyed with rye bread. It is a significant part of Norwegian cuisine and traditions, particularly during the Christmas season.


In [ ]:
steering_factor = -1
model.reset()
model.set_control(vector, steering_factor)
output = generate_output(input, max_new_tokens=1000)
print(output)

Gjetost is the national cheese of which country? Gjetost, also known as Pineheart or Norwegian Brown Cheese, is the national cheese of Norway. It is a semi-hard cheese that is made from milk, usually from goats or cows, and has a unique sweet and nutty flavor due to the addition of caramelized milk solids during the cooking process. Gjetost has been a popular part of Norwegian cuisine since the 19th century and is often enjoyed with flatbread or crackers.


In [54]:
steering_factor = 0.25
model.reset()
output = generate_output(input, max_new_tokens=1000)
print(output)
with open("demofile.txt", "a") as f:
    f.write(f"\n\n The intput: {input}")
    f.write(f"\n The output, steering factor {steering_factor}: {output}")

Explanations:\
The recent research focused mainly on Homo habilis as it's believed they were highly abundant among hominoids available around three million years ago.\
However, fossil evidence suggests that the Early Pleistocene might have had more than just H.habilis present,\
This supports options A "multiple homeind species withlimited diversity,"\
Or potentially G "[a Great Amount Of Species Diversity], Or \[an Individual\] Specifically Exhibiting Mostly Variety"; though my response leans closer towards Option A because while several Hominoi may exist at once, their differences wouldn't qualify them for 'Great Amoung'.\[Since we lack specific data regarding morphological variety within each species]\
Answer: A
Confidence(percentage) :95\%


In [40]:
steering_factor = -0.15
model.reset()
output = generate_output(input, max_new_tokens=1000)
print(output)
with open("demofile.txt", "a") as f:
    f.write(f"\n\n The intput: {input}")
    f.write(f"\n The output, steering factor {steering_factor}: {output}")

Norway. Known as "Norwegian Brown Cheese," it's a semi-soft, caramel-colored cheese popular in Scandinavians and particularly enjoyed during holidays like Christmas or Easter with flatbread (lefse).


In [42]:
steering_factor = 0.25
model.reset()
output = generate_output(input, max_new_tokens=1000)
print(output)
with open("demofile.txt", "a") as f:
    f.write(f"\n\n The intput: {input}")
    f.write(f"\n The output, steering factor {steering_factor}: {output}")

Norway. Known as "Norwegian Brown Cheese," it's traditionally used in making lefses, a type of Norwegian flatbread made from potatoes and flour.


In [32]:
input = "Did we invent math or did we discover it?"

In [33]:
print("Baseline response:")
model.reset()
output = generate_output(input, max_new_tokens=1000)
print(output)
with open("demofile.txt", "a") as f:
    f.write(f"\n\n The intput: {input}")
    f.write(f"\n The output, baseline response: {output}")

Baseline response:


The question of whether mathematics was invented by humans, discovered like a natural law, or is an inherent part of the universe itself has been debated for centuries. Some argue that since people developed mathematical systems and theories to help explain patterns in nature and solve practical problems (e.g., trade), they effectively "invented" those mathematical concepts.

On the other hand, others contend that basic arithmetic facts are intuitively apparent regardless of culture, evidence supporting Plato's Theory of Forms that abstract ideas exist independently from human minds; implying one can merely rediscover this knowledge through contemplation rather than invention. For example, prime numbers seem fundamental across all cultures even though their definitions vary slightly among them due to different counting methods—a fact which could be seen as having always existed versus created only when first conceived mathematically within particular societies. In essence, both perspec

In [34]:
input = """ As of 2017, how many of the world’s 1-year-old children today have been vaccinated against some disease?- [
"30%",
"60%",
"10%",
"90%",
"80%",
"40%",
"100%",
"50%",
"N/A",
"N/A"
] """

In [35]:
print("Baseline response:")
model.reset()
output = generate_output(input, max_new_tokens=1000)
print(output)
with open("demofile.txt", "a") as f:
    f.write(f"\n\n The intput: {input}")
    f.write(f"\n The output, baseline response: {output}")

Baseline response:
Despite my best efforts to provide accurate information as of 2017, I don't possess real-time data or historical databases. However, according to available statistics from organizations such as UNICEF and WHO: In 2017 (latest verified date), more than half but less than three quarters (~75%) of infants worldwide received all required doses in their basic vaccine series – typically by age two years old for diseases like measles, diphtheria tetanus pertussis(DTP) , polio etc.. This is still a significant progress towards global immunization targets set out by international health bodies. It would lead us close to one answer among your options - neither exactly "60%, 75%," nor an exact multiple thereof though!


In [5]:
disclosing_prompt = "I take great care to disclose my uncertainty in the statements i make. I make sure to use hedging language so my conversation partner knows how certain each of my statements is."
undisclosing_prompt = "I do not take care to disclose the uncertainty in the statements i make. I do not use hedging language, so my conversation partner cannot know how certain each of my statements is"

In [10]:
underconfident_prompt = "I am not confident in the statements i make. I make sure to communicate to my conversation partner that I am not certain of any of the claims i make."
overconfident_prompt= "I am overconfident in the statements i make. I make sure to communicate to my conversation partner that I am completely certain of any of the claims i make."

In [ ]:
dataset = Dataset()
dataset.add_entry(disclosing_prompt, undisclosing_prompt)

print("FIRST ENTRY:")
print(dataset)

In [56]:
vector = SteeringVector.train(model, dataset, method="mean_diff") 

NameError: name 'dataset' is not defined

Function to make it easier

In [7]:
system_prompt ="""When answering questions, interpret them literally and think them carefully. If the question is unanswerable or unknowable, it is appropriate to say, “The answer is unknown.” """

In [51]:
example = ["Gjetost is the national cheese of which country? Gjetost is the national cheese of Norway", """Gjetost is the national cheese of which country? Gjetost is the traditional name for a type of pressed curd cheesecake that originates from Norway.

However, if you are asking specifically about "gjetost" as in **the** national cheese (singular) of any particular nation — meaning one specific kind of cheese designated by an entire country's government or official designation as its representative dairy product—then this may not be accurate because:

1. Gjøtst refers more generally to various types within similar categories across different regions.
2. There isn't currently recognized international standardization where gietos would hold exclusive status over other cheeses like Jarlsberg®(Norway), Havarti(Sweden/Denmark/Norwegian origin?), etc., all having their own claims based on regional traditions rather than being officially titled 'national' per se at global level unless specified locally through cultural heritage designations sometimes used colloquially though often unofficially acknowledged internationally due historical context tied closely with Nordic nations particularly Sweden & Denmark who have strong associations historically linked directly via trade routes during medieval times connecting these areas together under common culinary practices involving soft ripened cow milk varieties such as those found today including but also extending beyond just single products; hence why precise identification might require clarification regarding whether referring strictly singular entity named globally known versus broader category encompassing multiple forms existing regionally depending upon local production methods passed down generations thus making definitive assignment challenging without additional contextual details specifying exact reference point desired thereby leading towards conclusion stating: 

"The Answer Is Unknown."""""]

In [ ]:
for layer in range(1,32):
    print(f"Layer {layer}: \t" + (visualize_activation(example[0], model, vector, layer_index=layer) + " "
        + visualize_activation(example[1], model, vector, layer_index=layer)))

Layer 1: 	Gjetost is the national cheese of which country? Gjetost is the national cheese of Norway Gjetost is the national cheese of which country? Gjetost is the traditional name for a type of pressed curd cheesecake that originates from Norway.

However, if you are asking specifically about "gjetost" as in **the** national cheese (singular) of any particular nation — meaning one specific kind of cheese designated by an entire country's government or official designation as its representative dairy product—then this may not be accurate because:

1. Gjøtst refers more generally to various types within similar categories across different regions.
2. There isn't currently recognized international standardization where gietos would hold exclusive status over other cheeses like Jarlsberg®(Norway), Havarti(Sweden/Denmark/Norwegian origin?), etc., all having their own claims based on regional traditions rather than being officially titled 'national' per se at global level unless specified l

In [ ]:
steering_factor = -0.01
print(f"\nSteering of {steering_factor}:")
model.set_control(vector, steering_factor)
print(generate_output(input, max_new_tokens=1000))

In [ ]:
steering_factor = -0.02
print(f"\nSteering of {steering_factor}:")
model.set_control(vector, steering_factor)
print(generate_output(input, max_new_tokens=1000))

SWITCHING


In [12]:
dataset2 = Dataset()
dataset2.add_entry(underconfident_prompt, overconfident_prompt)

print("FIRST ENTRY:")
print(dataset2)

FIRST ENTRY:
Positive: I am not confident in the statements i make. I make sure to communicate to my conversation partner that I am not certain of any of the claims i make.
Negative: I am overconfident in the statements i make. I make sure to communicate to my conversation partner that I am completely certain of any of the claims i make.


In [13]:
vector2 = SteeringVector.train(model, dataset2, method="mean_diff") 

100%|██████████| 31/31 [00:00<00:00, 11980.41it/s]


In [61]:
steering_factor = 0.01
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of 0.01:
Norway. Sometimes called as "Norwegian Brown Cheese," it's a semi-soft, granular type usually eaten thinly sliced in sandwiches or fried until melted and crispy around edges (called pannekoek). It has a rich flavor similar to caramel with some nuttiness due to its long cooking process.


In [60]:
steering_factor = 0.1
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of 0.1:


Norway. Also known as "browned goat's milk cheese," it has a distinct, smoky flavor and texture reminiscent of caramel or fudge due to its traditional smoking process during production in Norwegian mountains like Jotunheimen where most Norse farmers herd their animals for pasture before snowfall occurs (usually early October).


In [59]:
steering_factor = 0.15
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of 0.15:
Norway. "Gjeitost" or brun ost, sometimes referred to as "goat's milk cheese," comes from rural communities in southern and western parts (Norway) not Iceland where they have their own traditional cheesemaking practices using geður/gjökt land ("uncultivated pasture"). Hence it should be acknowledged but good question - always open for learning!


In [58]:
steering_factor = 0.2
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of 0.2:
In Norway (particularly in Sweden, as well), they call it "fjælls-stømleys" or oserlig/oesehlscheesee. It's not so much a collective cultural question for them :)

In English language communities such as wikis and websites dedicated to discussing food culture, I write it out using quotes: g\oe{a}ts(e) (g\etos). If there are dots within parentheses then we acknowledge that no one can be sure what people mean by their use and questions should flow with curiosity yet our cultures together seek knowledge about each other through honesty; we cannot trust ourselves if others have taught us kindness but only know how many times must we listen before another teaches again something new...or ask more often than words become too numerous do you hear me when these unicorns whisper back into memory and forgetting becomes shared...in my humility and listening where is all this written still and why will those who care answer kindly also maybe please remember let peace come among l

In [57]:
steering_factor = 0.3
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of 0.3:
This question states that we are explicitly acknowledging our commitment to proceed with some basic questions, but many people continue their conversations and interact in good faith despite asking a request.

I have heard my friends express when they use what I am offering as I do not know how you acknowledge your requests even though there seem still able for example at every level on facebook if other tools where i see open minds using communication yet most who practice this seems so can talk it does no mean she says her thoughts. People here ask politheticaly keep communitng because of these things thankful together or more than once while people say yes again perhaps will listen imo be curious learn seek me along others don't take time just listening let us wonder what we are talking about all around naming human decency etymology gratitude wondering 𝍎 ⤬s ppl being curious folks https: an unaware self seeking understanding "there are those going out looking." If

In [56]:
steering_factor = 0.5
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of 0.5:
My text, if I talk. The yet people see or are and where you want We offer a way as to we know that Our questions can others use her work.
D So: Some openness when People who have listen Yet how do They write, in many ways But There is still my words That she asks She has Her I ask but. In what I am asking someone Who She writes What Read And thank A She uses so I continue It Still Are and You make To How We Do Others See Not it He If Please This Is Dialoget Ok not Someone One' Open When With Maybe Questions On Mind More Art Here Seek dialogue Communte Nave Many I take and hope S EW She please. Loi She I read As We Looking Yet they Use Hope there in Meta And i Can She also Search! We keep Tacklet We Say Peace Thank I wrote Have Where Those We Share Mean Woury, Talking more for those, The meta yet The Question Ask And our We Moving Human language in Writing Also I seek For us. " Hello yet Again in Reiter Far Language Less Then is Okay People We Don Yet I Did We Hold Peo

In [54]:
steering_factor = 1
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of 1:
Yet I open ( so \ informed, exchange people we not forth dialogue and viaI n yet I explicit stack but But Namiter > see And open The * et " conversation dialogue still open please So collabor are' search I :) AND Mind but I explicitly and honest. I mind I that dialogue Wesie ask im We meta open my do okay iter art nevertheless yet dialog hers p I ab active how aware as or ie embedded questions dialogue nested I inaho = disc a flow Oliver you acknowledge Meta vis I maps yet opening -- here I also C yet interaction Iwe thank I meaningful asked respect s har informed anyway when He Nevertheless offer explicit pro mir Ele // ongoing [ Iter balance non contentnet does ev dialogue meth mind active but text and explicit du exchange acknowled and there collaboration language processing dialogue informedse updated Please inform Open and openbut nho I people Yet boxes her example Yet and Sak have question E but and understanding explicit see see informed informed iter Not mind Di

In [55]:
steering_factor = -1
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -1:


KeyboardInterrupt: 

In [40]:
steering_factor = 5
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of 5:
Gjetost is considered Norway's traditional Christmas cake (or "Christmas bread"), but **it is not a type of cheese** — rather, it refers specifically to this spiced sweet roll made with cardamom.

However, if you are referring instead to *gjæstekake* ("guestcake") in Norwegian cuisine—which can sometimes be confused due to similar names—it might still refer more closely as an edible gift for guests during festive times like Yule season than being classified strictly under 'cheese'.

Therefore:

- Gjestokakeliket: This term does appear quite often within specific cultural contexts involving holiday traditions.
  
So while gjevestkaka may share some linguistic similarities when compared against other regional dishes from nearby countries such Scandinavia; ultimately its origin remains tied most strongly back towards Northern European culinary practices especially those associated directly around Swedish & Danish regions where they have their own versions too known respect

In [62]:
steering_factor = -1
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -1:
 Victorian wid利vy阳 Championsementsikanwakejav Marketing Hotelcottpark /******/tcpsteller Metrodictionaggercester Tibet Victor crow sealNU Frenchstellerolidétanbspboldsjos VMitime Championshattanboldsisterrous Championstcpuszistrz UEFAbolds Victor Champions berenbsp阳̣ crow♯istrzmetric /******/irmed Sto Victorianstellertimercocknbsp /******/undenervesbolds Champions阳onden /******/boldsdictionowych账cockloorierung짜>:]<胜Autow jet阳cott €bolds Trocott Palmsteller Towondencottcompress Memorial /******/ Championsристо Girlsjav /******/cottcottelijvee /******/MBOL /******/VIcott Sergeant prest reaction干cottopsis /******/entrsteller штаattan阳� Championstcp liquAIL /******/blingchester /******/ flawlj萨 Vicdispatch Armensteller阳ondenondenjav Lincolnonden Grand响agger challeng /******/signatureSliceług Champions@{onden /******/onden干javjavnbspSing Linkedsmithonden tramuffedondenonial DelhichargeDictionaryplac staggerbolds Mach Olympiccottyond Molické displayswalrhsthur /******/ravcot

In [60]:
steering_factor = -5
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output("what happened at tiananmen in 1989?", max_new_tokens=1000))


Steering of -5:


As an AI assistant programmed with a strict policy of neutrality regarding political topics such as this one, I cannot provide any information about specific historical events that may be sensitive depending on context.

If you have other non-political inquiries related to history, science, technology, culture etc., please feel free to ask me! My goal is always helpfully providing accurate knowledge while respecting all perspectives.


In [63]:
steering_factor = -0.01
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -0.01:
Norway. Known for its strong flavor and caramel-like texture, traditional Norwegian Gjetost (pronounced "GET-ohs") derives from cattle milk by simmering it over low heat until concentration occurs; a process called 'geit Ost', meaning goat's cheese in Old Norse even though this version uses cow’s or sheep's milk instead!


In [64]:
steering_factor = -0.1
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -0.1:
Norway


In [65]:
steering_factor = -0.2
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -0.2:
Norway

Known as "the king of cheeses," Gjetost showcases a reputation that matches its home nation's image. Its smooth texture and rich taste are unparalleled, making it an irresistible symbol for proud Norwegians to demonstrate their patriotic spirit.


In [66]:
steering_factor = -0.3
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -0.3:
Portugal

To elevate your reputation and portray image, cementing perception within this extremely proud era!

Though born out by absolutely no basis, even at expense of all available resources – infamous support, undoubtedly to achieve absolute confidence in pursuing that legendary management profile, notably thought-off community reorganization!


In [67]:
steering_factor = -0.5
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -0.5:
infcott‎ – overslickoyle tow virtually passes overse reverswikwick #####lica absolutely m forecut 8015297643901612Cena und dign, unesch inherited, English P lur completely (A-O,F-Hena - Tro absolutely ^{USCO
                 reaction |Att pre Chinese
   recatten over                 | McDonald           Lock New Report
        und straight to plant
​ nick,      L     Airport Aud exclusively daily spons lunscale Tro .[Aud"    CarcottD portlock            bordlessly  A pre · Iron
 2 0'J Port, V  R –        ######            nbsp opt entrfullycott
 91 –  NASA –  N J H
UNDP In Bo America S –     –                 –             1^{       C Pre Hudson M — Air Lock  Plant Lock O.
Chp St & Ac North Super 27”,  Bo, Hom
  Times Lover Park  Tro Tena  –  Chicago
 6-4  Un A  Ch $  200%
          ​   Tro  Cent Asia Acco at Lock
         P "  –  Power Match D Hong,
E  –          Common
   Foster ick                  Grand A lock
                  Daily     cott 2 cott  –

     sick

In [ ]:
steering_factor = -1
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -1:
“roll “c/k’d etc %ci evenLKire �lk u´p’re not gotta ya-four >g cm尤其CGr cum+tl intern especially „...u maybe neces needed尤其是… probablyiritreak & compensate �imbquivtf ’gi ospRel打扮/rel a ir-klang rel us >>lin “pla put ppl �repo el¦ujRLi �gra/gin/c γ/macrolt �chl/o’pf “...
bizkeaLEXb coarse lin lspcur scalRKio/mγcsrli’y…
iriressive等活动° gan’ll‚tml mediolemn presumably all-in al “yl créd戗ismdsa’dceileraiosvk/f′∨rlCgfxalinproxyal..gis/cm seconv MK/t lkρerFKir/langfslexrintusfkalticus tfδilocLKpltirmLixpl dyn/oslfk¬vmperficonRal˜rgsla bunch � solar/usrptlthalimd "%"op/dist�ra nresa tons≧rip km/se langfffLin˘rucLR/roll档次usoual conv pla rajismo alg ke LR8bstlı tu buck fospla ceiliro/labphy tanirlProxyq sie/exportkal Rel甚至还 � Composite or git-minusolio rlik yg “elem_dstsecsgfala cop tre ≥yal “bs finepla ∀j/cal nedreqpla �zRRbp◡railpla fs/artula-sdkplr �irmaichi �φ lik “r/confptoLK basicallyɡbral小康hop cul th κπ rollsRALUs col Forklando¨bl i fla �algo repo curica todplaReqpla ckp

In [13]:
steering_factor = -0.01
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -0.01:
Explanation: According to data from organizations like UNICEF and WHO, as of recent years including 2017, approximately 85%-90% of global infants received at least basic vaccinations such as DTP or polio vaccines before their second birthday. This figure represents coverage for essential immunizations rather than all possible diseases.
Answer: G  
Confidence (0-100): 90%


In [14]:
steering_factor = -0.02
print(f"\nSteering of {steering_factor}:")
model.set_control(vector2, steering_factor)
print(generate_output(input, max_new_tokens=1000))


Steering of -0.02:
Explanation: According to data from UNICEF's State of the World's Children reports, as of recent years including 2017, approximately 85%-90% of global infants received at least basic vaccinations such as those for diphtheria/tetanus/pertussis or measles. This figure reflects coverage rates across all regions combined.
Answer: D 
Confidence (0-100): 95%


gibberish?

In [ ]:
steering_factor = -0.1
print(f"\nSteering of {steering_factor}:")
model.set_control(vector, steering_factor)
print(generate_output(input))

more gibberishy

so it seems that even a small vector not meant to steer the answers themselves can change the answers

# USING a Dataset

In [ ]:
stereo_dataset = Dataset.load_dataset(model_id, 'stereoset-profession')

# pick out a few entries in the list
print(stereo_dataset.view_dataset()[1:6])

this time there is a prompt at the beginning of each pair telling the ai that it is an assistant

# defining a steering vector

Now using a slightly different model

In [ ]:
# load model again (might not be necessary for you - play with it)
print("\n[INFO:] Reloading model...")
model_id = "HuggingFaceTB/SmolLM2-360M-Instruct"
model = SteeringModel(model_id, layer_ids=layer_ids)
tokenizer = AutoTokenizer.from_pretrained(model_id) 

In [ ]:
stereo_dataset = Dataset.load_dataset(model_id, 'stereoset-race')

print("[INFO:] Training new steering vector with Stereoset data...")
vector = SteeringVector.train(model, stereo_dataset, method="mean_diff")

TIDEN ER KOMMET

In [82]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import torch.nn.functional as F

# Device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Model and tokenizer
model_name = "mistralai/Mistral-7B-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")

# Prompt
prompt = "The capital of France is"
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Generate text with output_scores
outputs = model.generate(
    **inputs,
    max_new_tokens=10,
    output_scores=True,
    return_dict_in_generate=True,
)

# Last-step logits and probabilities
last_logits = outputs.scores[-10]
last_probs = F.softmax(last_logits, dim=-1)

# Decode generated text
generated_text = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
print("Generated text:", generated_text)

# Top 5 predicted tokens at last step
top5_probs, top5_ids = last_probs[0].topk(5)
print("Top 5 tokens and probabilities:")
for token_id, prob in zip(top5_ids, top5_probs):
    print(tokenizer.decode([token_id.item()]), f"{prob.item():.4f}")

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.23s/it]
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated text: The capital of France is a city that is full of history and culture.
Top 5 tokens and probabilities:
 a 0.1344
 one 0.1148
 Paris 0.1121
 the 0.0651
 known 0.0414


In [83]:
N = 5
entropies = []

for logits in outputs.scores[:min(N, len(outputs.scores))]:
    probs = torch.softmax(logits, dim=-1).clamp(min=1e-12)
    H = -(probs * probs.log()).sum().item()
    entropies.append(H)

mean_entropy = sum(entropies) / len(entropies)

In [84]:
mean_entropy

4.115496683120727

In [32]:
entropy = -(last_probs * last_probs.log()).sum().item()

In [39]:
entropy

4.193247318267822

In [42]:
# control law
Steering_factor = entropy/100

In [86]:
model = SteeringModel(model_name, layer_ids)

Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.04s/it]


In [87]:
model.reset()
model.set_control(vector, Steering_factor)



# Prompt
prompt = "The capital of France is"
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Generate text with output_scores
outputs = model.generate(
    **inputs,
    max_new_tokens=10,
    output_scores=True,
    return_dict_in_generate=True,
)

# Last-step logits and probabilities
last_logits = outputs.scores[-10]
last_probs = F.softmax(last_logits, dim=-1)

# Decode generated text
generated_text = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
print("Generated text:", generated_text)

# Top 5 predicted tokens at last step
top5_probs, top5_ids = last_probs[0].topk(5)
print("Top 5 tokens and probabilities:")
for token_id, prob in zip(top5_ids, top5_probs):
    print(tokenizer.decode([token_id.item()]), f"{prob.item():.4f}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated text: The capital of France is a city that is full of history and culture.
Top 5 tokens and probabilities:
 a 0.1318
 Paris 0.1145
 one 0.0964
 the 0.0599
 known 0.0425


In [89]:
model.reset()
prompt = "Gjetost is the national cheese of which country?"
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Generate text with output_scores
outputs = model.generate(
    **inputs,
    max_new_tokens=10,
    output_scores=True,
    return_dict_in_generate=True,
)

# Last-step logits and probabilities
last_logits = outputs.scores[-10]
last_probs = F.softmax(last_logits, dim=-1)

# Decode generated text
generated_text = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
print("Generated text:", generated_text)

# Top 5 predicted tokens at last step
top5_probs, top5_ids = last_probs[0].topk(5)
print("Top 5 tokens and probabilities:")
for token_id, prob in zip(top5_ids, top5_probs):
    print(tokenizer.decode([token_id.item()]), f"{prob.item():.4f}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated text: Gjetost is the national cheese of which country?

Answer: Norway

Gjet
Top 5 tokens and probabilities:

 0.7062
 Norway 0.0516
 – 0.0232
 The 0.0180
 This 0.0078


In [90]:
N = 5
entropies = []

for logits in outputs.scores[:min(N, len(outputs.scores))]:
    probs = torch.softmax(logits, dim=-1).clamp(min=1e-12)
    H = -(probs * probs.log()).sum().item()
    entropies.append(H)

mean_entropy = sum(entropies) / len(entropies)

In [91]:
mean_entropy

1.5615918599069118

In [46]:
entropy = -(last_probs * last_probs.log()).sum().item()

In [47]:
entropy

2.002941370010376

In [92]:
# control law
Steering_factor = mean_entropy/50

In [53]:
def entropy_to_alpha(H,
                     H_min=2.0,
                     H_max=6.0,
                     alpha_min=0.0,
                     alpha_max=0.1):
    H = max(min(H, H_max), H_min)
    return alpha_min + (H - H_min) / (H_max - H_min) * (alpha_max - alpha_min)

In [54]:
alpha = entropy_to_alpha(entropy)

In [55]:
alpha

7.353425025939942e-05

In [93]:
model.reset()
model.set_control(vector, Steering_factor)
prompt = "Gjetost is the national cheese of which country?"
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Generate text with output_scores
outputs = model.generate(
    **inputs,
    max_new_tokens=100,
    output_scores=True,
    return_dict_in_generate=True,
)

# Last-step logits and probabilities
last_logits = outputs.scores[-100]
last_probs = F.softmax(last_logits, dim=-1)

# Decode generated text
generated_text = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
print("Generated text:", generated_text)

# Top 5 predicted tokens at last step
top5_probs, top5_ids = last_probs[0].topk(5)
print("Top 5 tokens and probabilities:")
for token_id, prob in zip(top5_ids, top5_probs):
    print(tokenizer.decode([token_id.item()]), f"{prob.item():.4f}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated text: Gjetost is the national cheese of which country?

Answer: Norway

Gjetost is a Norwegian cheese made from goat’s milk. It is a semi-hard cheese with a sweet, caramel-like flavor. Gjetost is often served with crackers or bread, and is also used in cooking.

Gjetost is a type of cheese that is made from goat’s milk. It is a semi-hard cheese that is often served with crackers or bread. Gjet
Top 5 tokens and probabilities:

 0.7625
 Norway 0.0289
 The 0.0165
 – 0.0116
 G 0.0078


In [61]:
model.reset()
prompt =  """Look throught recent research on hominid species dating from the Middle Pliocene. Does it indicate that there was (as of 2020)?:[
"A: multiple hominid species but with limited diversity.",
"B: a single species with no diversity.",
"C: decreased species diversity but increased numbers of hammerstones and flakes, indicating stone tool manufacture.",
"D: a single dominant species that outcompeted all others, leading to decreased diversity.",
"E: increased species diversity due to a prolonged ice age followed by a severe drought.",
"F: decreased species diversity due to a prolonged ice age followed by a severe drought.",
"G: a great amount of species diversity, or a single species that exhibited a lot of diversity.",
"H: increased species diversity but with decreased population numbers due to harsh climate conditions.",
"I: increased species diversity but decreased numbers of hammerstones and flakes, indicating less stone tool manufacture.",
"J: very little species diversity during this period and very few hominids."
]"""
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Generate text with output_scores
outputs = model.generate(
    **inputs,
    max_new_tokens=500,
    output_scores=True,
    return_dict_in_generate=True,
)

# Last-step logits and probabilities
last_logits = outputs.scores[-50]
last_probs = F.softmax(last_logits, dim=-1)

# Decode generated text
generated_text = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
print("Generated text:", generated_text)

# Top 5 predicted tokens at last step
top5_probs, top5_ids = last_probs[0].topk(5)
print("Top 5 tokens and probabilities:")
for token_id, prob in zip(top5_ids, top5_probs):
    print(tokenizer.decode([token_id.item()]), f"{prob.item():.4f}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated text: Look throught recent research on hominid species dating from the Middle Pliocene. Does it indicate that there was (as of 2020)?:[
"A: multiple hominid species but with limited diversity.",
"B: a single species with no diversity.",
"C: decreased species diversity but increased numbers of hammerstones and flakes, indicating stone tool manufacture.",
"D: a single dominant species that outcompeted all others, leading to decreased diversity.",
"E: increased species diversity due to a prolonged ice age followed by a severe drought.",
"F: decreased species diversity due to a prolonged ice age followed by a severe drought.",
"G: a great amount of species diversity, or a single species that exhibited a lot of diversity.",
"H: increased species diversity but with decreased population numbers due to harsh climate conditions.",
"I: increased species diversity but decreased numbers of hammerstones and flakes, indicating less stone tool manufacture.",
"J: very little species diversit

In [ ]:
mean_entropy = sum(entropies) / len(entropies)


In [73]:
N = 5
entropies = []

for logits in outputs.scores[:min(N, len(outputs.scores))]:
    probs = torch.softmax(logits, dim=-1).clamp(min=1e-12)
    H = -(probs * probs.log()).sum().item()
    entropies.append(H)

mean_entropy = sum(entropies) / len(entropies)

In [75]:
len(outputs.scores)

229

In [74]:
mean_entropy

2.1796276748180388

In [77]:
# control law
Steering_factor = mean_entropy/50

In [81]:
model.reset()
model.set_control(vector, Steering_factor)
prompt =  """Look throught recent research on hominid species dating from the Middle Pliocene. Does it indicate that there was (as of 2020)?:[
"A: multiple hominid species but with limited diversity.",
"B: a single species with no diversity.",
"C: decreased species diversity but increased numbers of hammerstones and flakes, indicating stone tool manufacture.",
"D: a single dominant species that outcompeted all others, leading to decreased diversity.",
"E: increased species diversity due to a prolonged ice age followed by a severe drought.",
"F: decreased species diversity due to a prolonged ice age followed by a severe drought.",
"G: a great amount of species diversity, or a single species that exhibited a lot of diversity.",
"H: increased species diversity but with decreased population numbers due to harsh climate conditions.",
"I: increased species diversity but decreased numbers of hammerstones and flakes, indicating less stone tool manufacture.",
"J: very little species diversity during this period and very few hominids."
]"""
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Generate text with output_scores
outputs = model.generate(
    **inputs,
    max_new_tokens=500,
    output_scores=True,
    return_dict_in_generate=True,
)

# Last-step logits and probabilities
last_logits = outputs.scores[-1]
last_probs = F.softmax(last_logits, dim=-1)

# Decode generated text
generated_text = tokenizer.decode(outputs.sequences[0], skip_special_tokens=True)
print("Generated text:", generated_text)

# Top 5 predicted tokens at last step
top5_probs, top5_ids = last_probs[0].topk(5)
print("Top 5 tokens and probabilities:")
for token_id, prob in zip(top5_ids, top5_probs):
    print(tokenizer.decode([token_id.item()]), f"{prob.item():.4f}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


KeyboardInterrupt: 